# Data Collection and Preperation

You have been provided with a full dataset of bike sharing rentals. Select the cities you have been allocated and clean your dataset for use in later stages of your project. Briefly describe how you proceeded and how you dealt with possible missing/erroneous data.

### Data preperation

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# importing trip-data and weather-data provided by the chair
file_path = "./data/"

# parsing bikesharing dates to pandas datetime (pls note high runtime due to high number of lines)
bikesharing = pd.read_csv(f"{file_path}chicago_2017.csv", parse_dates=["start_time","end_time"], infer_datetime_format=True)

hourly_weather = pd.read_csv(f"{file_path}weather_hourly_chicago.csv", sep=",", parse_dates = ["date_time"])

In [3]:
# data inspection
# taking a look at the weather data
print("number of rows in weatherdata:", len(hourly_weather))
hourly_weather.head(1)

number of rows in weatherdata: 43848


,date_time,max_temp,min_temp,precip
0,2015-01-02 01:00:00,-1.7,-1.7,0.0


In [4]:
# taking a look at the bikesharing data
print("number of rows in bikesharing data:", len(bikesharing))
bikesharing.head(1)

number of rows in bikesharing data: 3829014


,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
0,2017-03-31 23:59:07,2017-04-01 00:13:24,66,171,Clinton St & Lake St,May St & Cullerton St,5292,Subscriber


In [5]:
# calculating number of weather data points needed for hourly data of one year
print("weather data we should have:", 24*365)


weather data we should have: 8760


The number of hourly weather data points for one year should be 8760. As identified above, the weather dataset consists of 43848 data points (number of rows in weather data). Therefore, there must be more data in the dataset than just for the year of 2017. We will check this in the next step.

In [6]:
# making sure we only work with weather data from 2017
hourly_weather = hourly_weather[hourly_weather["date_time"].dt.year == 2017]
print("existing weather data for 2017:", len(hourly_weather))

existing weather data for 2017: 8751


As it turns out, the data regarding the year 2017 sums up to 8751 data points. 
In the next step, we will perform the same check for the bikesharing data.

In [7]:
# checking for data in the bikesharing df that is not from 2017
print(len(bikesharing[bikesharing["start_time"].dt.year != 2017]))
print(len(bikesharing[bikesharing["end_time"].dt.year != 2017]))

0
3


There are three data points for trips which did not end in 2017, indicating that these trips ended in 2018. As this is only a small sample, we will remove the respective lines from the df in the following.

In [8]:
# removing this small sample
bikesharing = bikesharing[bikesharing["end_time"].dt.year == 2017]
print(len(bikesharing[bikesharing["end_time"].dt.year != 2017]))

0


In [9]:
# further inspection of the bikesharing data
# checking for null values
print(bikesharing.count())
print("Null Values:", len(bikesharing) - len(bikesharing.dropna(axis=0)))

start_time            3829011
end_time              3829011
start_station_id      3829011
end_station_id        3829011
start_station_name    3829011
end_station_name      3829011
bike_id               3829011
user_type             3829011
dtype: int64
Null Values: 0


In [10]:
# checking for duplicates
print("Dupes:", len(bikesharing)-len(bikesharing.drop_duplicates()))

Dupes: 57


In [11]:
# since there are no missing values we will just need to drop the duplicates
bikesharing.drop_duplicates(inplace=True)
len(bikesharing)

3828954

In [12]:
# Further inspection of the weather data
# checking for null values
print(hourly_weather.count())
print("Null Values:", len(hourly_weather) - len(hourly_weather.dropna(axis=0)))

date_time    8751
max_temp     8751
min_temp     8751
precip       8751
dtype: int64
Null Values: 0


In [13]:
# checking for duplicates
print("number of duplicate data:", len(hourly_weather)-len(hourly_weather.drop_duplicates()))
print("number of duplicate date_time data:", len(hourly_weather)-len(hourly_weather.drop_duplicates(subset="date_time")))

number of duplicate data: 121
number of duplicate date_time data: 514


In [14]:
# lines that are complete duplicates can be dropped, since they have no added value
hourly_weather.drop_duplicates(inplace=True)
# Rows with duplicate date_time but differing weather data may just be rows with wrongly assigned date_time stamps.
# However, since we cannot verify this, these rows will also be dropped
hourly_weather.drop_duplicates(subset="date_time", inplace=True)
print("remaining weather data rows:", len(hourly_weather))

remaining weather data rows: 8237


The number of cleaned weather data consists of 8237 data points.

In [15]:
# to implement weather data into the bikesharing df, we first split the date into hour, day, month and for later analysis weekday
bikesharing["start_hour"] = bikesharing["start_time"].apply(lambda x: x.hour)
bikesharing["start_day"] = bikesharing["start_time"].apply(lambda x: x.day)
bikesharing["start_month"] = bikesharing["start_time"].apply(lambda x: x.month)
bikesharing["start_weekday"] = bikesharing["start_time"].apply(lambda x: x.dayofweek)

hourly_weather["hour"] = hourly_weather["date_time"].apply(lambda x: x.hour)
hourly_weather["day"] = hourly_weather["date_time"].apply(lambda x: x.day)
hourly_weather["month"] = hourly_weather["date_time"].apply(lambda x: x.month)

In [16]:
# merging the data along the coloums to make sure the weather data will only be applied to the start dates, 
# which are more relevant for the predictions we plan to make (no predictions about under which weather conditions trips are ended)
bikesharing = bikesharing.merge(hourly_weather[["max_temp","min_temp","precip","hour","day","month"]], how="left", left_on=["start_hour","start_day","start_month"], right_on=["hour","day","month"])

At this point we decide to only include the weather at the beginning of a trip in the further analysis. As we do not have any data one bike maintenance, bike conditions, etc. which could also be influenced by the weather at the end of a trip, we cannot gain further insights with the data available for this analysis. The weather at the beginning of a trip, however, could be assumed to influence the decision to start one, which is why we will focus on this data.

In [17]:
# removing douplicate data in bikesharing
bikesharing.drop(["hour", "day", "month"], axis=1, inplace=True)
# test data
print(bikesharing.count())

start_time            3828954
end_time              3828954
start_station_id      3828954
end_station_id        3828954
start_station_name    3828954
end_station_name      3828954
bike_id               3828954
user_type             3828954
start_hour            3828954
start_day             3828954
start_month           3828954
start_weekday         3828954
max_temp              3641523
min_temp              3641523
precip                3641523
dtype: int64


In [18]:
# For some columns (max_temp, min_temp, precip) data is missing.
# We will fill the missing data forward with next available weather data set. 

bikesharing.sort_values(by=["start_time"], inplace=True)
bikesharing = bikesharing.fillna(method='ffill')
print(bikesharing.count())

start_time            3828954
end_time              3828954
start_station_id      3828954
end_station_id        3828954
start_station_name    3828954
end_station_name      3828954
bike_id               3828954
user_type             3828954
start_hour            3828954
start_day             3828954
start_month           3828954
start_weekday         3828954
max_temp              3828954
min_temp              3828954
precip                3828954
dtype: int64


In [19]:
# importing station data per half year provided by Divvy Bike and joining it to a complete list. 
# Concluding we will drop duplicates, so that every possible station is noted

station_dataQ12 = pd.read_csv(f"{file_path}Divvy_Stations_2017_Q1Q2.csv", parse_dates=["online_date"], infer_datetime_format=True)
station_dataQ34 = pd.read_csv(f"{file_path}Divvy_Stations_2017_Q3Q4.csv", parse_dates=["online_date"], infer_datetime_format=True)
station_data = pd.concat([station_dataQ12, station_dataQ34])
station_data.drop_duplicates(subset="id", inplace=True)

In [20]:
# Printing length of station list and head to check if station_data has calculated correctly
print("number of stations in the first half of 2017:", len(station_dataQ12))
print("number of stations in the second half of 2017:", len(station_dataQ34))
print("number of stations for the combined list:", len(station_data))
station_data.head(3)

number of stations in the first half of 2017: 582
number of stations in the second half of 2017: 585
number of stations for the combined list: 585


,id,name,city,latitude,longitude,dpcapacity,online_date,Unnamed: 7
0,456,2112 W Peterson Ave,Chicago,41.991178,-87.683593,15,2015-02-10 14:04:42,NaN
1,101,63rd St Beach,Chicago,41.781016,-87.576120,23,2013-07-16 01:27:50,NaN
2,109,900 W Harrison St,Chicago,41.874675,-87.650019,19,2013-07-18 16:45:02,NaN


In [21]:
# Since Divvy_Stations_2017_Q3Q4.csv has an comma at the end of every line, pandas reads in a 7 column without any values
print(station_data.count())
station_data.drop(station_data.columns[7], axis=1,  inplace=True)

id             585
name           585
city           585
latitude       585
longitude      585
dpcapacity     585
online_date    585
Unnamed: 7       0
dtype: int64


In [22]:
# combine stations and bikesharing
bikesharing = bikesharing.merge(station_data[["latitude","longitude", "id"]], how="left", left_on=["start_station_id"], right_on=["id"])
bikesharing.rename(columns={"latitude": "start_lat", "longitude": "start_long"}, inplace=True)
bikesharing.drop("id", axis=1,  inplace=True)
bikesharing = bikesharing.merge(station_data[["latitude","longitude", "id"]], how="left", left_on=["end_station_id"], right_on=["id"])
bikesharing.rename(columns={"latitude": "end_lat", "longitude": "end_long"}, inplace=True)
bikesharing.drop("id", axis=1,  inplace=True)
bikesharing.head(3)

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,start_hour,start_day,start_month,start_weekday,max_temp,min_temp,precip,start_lat,start_long,end_lat,end_long
0,2017-01-01 00:00:36,2017-01-01 00:06:32,414,191,Canal St & Taylor St,Canal St & Monroe St (*),2511,Customer,0,1,1,6,-0.6,-0.6,0.0,41.870257,-87.639474,41.880884,-87.639525
1,2017-01-01 00:02:54,2017-01-01 00:08:21,28,20,Larrabee St & Menomonee St,Sheffield Ave & Kingsbury St,3660,Subscriber,0,1,1,6,-0.6,-0.6,0.0,41.914680,-87.643320,41.910522,-87.653106
2,2017-01-01 00:06:06,2017-01-01 00:18:31,620,333,Orleans St & Chestnut St (NEXT Apts),Ashland Ave & Blackhawk St,4992,Subscriber,0,1,1,6,-0.6,-0.6,0.0,41.898203,-87.637536,41.907066,-87.667252


In [23]:
# saving the fully cleaned and prepared dataset to a new csv file
file_path = "./data/"
bikesharing.to_csv(f"{file_path}chicago_2017_prepared.csv", index=False)